# Chuẩn hóa dữ liệu thời khóa biểu và phân công giảng viên

Notebook này được thiết kế để:
1. Chuẩn hóa dữ liệu thời khóa biểu từ file CSV
2. Phân công giảng viên phù hợp dựa trên khoa/bộ môn
3. Đảm bảo không có xung đột về thời gian giảng dạy
4. Loại bỏ các môn học không cần phân công giảng viên (DA, TTTN, KLTN, PE231)

## Quy tắc phân công:
- Môn học thuộc khoa/bộ môn nào thì giảng viên khoa ấy dạy
- Giảng viên không thể dạy các lớp khác nhau trong cùng một tiết học
- Phòng học/lịch học có dấu "*" được giữ nguyên (online/không phân chia)
- Loại trừ: DA, TTTN, KLTN, PE231
- **Các môn bắt đầu bằng NT (như NT101, NT102...) thuộc khoa MMTTT và được phân công giảng viên bình thường**

In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Thư viện đã được import thành công!")

Thư viện đã được import thành công!


## 1. Load và Explore Data

Tải dữ liệu từ các file CSV và khám phá cấu trúc dữ liệu.

In [3]:
# Load dữ liệu từ các file CSV
print("Đang tải dữ liệu...")

# Đường dẫn đến các file dữ liệu
schedule_path = r"d:\eUIT\scripts\database\other_data\thoi_khoa_bieu.csv"
teachers_path = r"d:\eUIT\scripts\database\main_data\giang_vien.csv"
subjects_path = r"d:\eUIT\scripts\database\main_data\danh_muc_mon_hoc.csv"

# Load dữ liệu thời khóa biểu
df_schedule = pd.read_csv(schedule_path, sep=';', encoding='utf-8')
print(f"Đã tải {len(df_schedule)} dòng dữ liệu thời khóa biểu")

# Load dữ liệu giảng viên
df_teachers = pd.read_csv(teachers_path, encoding='utf-8')
print(f"Đã tải {len(df_teachers)} giảng viên")

# Load dữ liệu môn học
df_subjects = pd.read_csv(subjects_path, sep=';', encoding='utf-8')
print(f"Đã tải {len(df_subjects)} môn học")

print("\nTải dữ liệu hoàn tất!")

Đang tải dữ liệu...
Đã tải 824 dòng dữ liệu thời khóa biểu
Đã tải 242 giảng viên
Đã tải 961 môn học

Tải dữ liệu hoàn tất!


In [16]:
# Khám phá cấu trúc dữ liệu
print("=== CẤU TRÚC DỮ LIỆU THỜI KHÓA BIỂU ===")
print("Columns:", list(df_schedule.columns))
print("Shape:", df_schedule.shape)
print("\nSample data:")
print(df_schedule.head())

print("\n=== CẤU TRÚC DỮ LIỆU GIẢNG VIÊN ===")
print("Columns:", list(df_teachers.columns))
print("Shape:", df_teachers.shape)
print("\nKhoa/Bộ môn của giảng viên:")
print(df_teachers['khoa_bo_mon'].value_counts())

print("\n=== CẤU TRÚC DỮ LIỆU MÔN HỌC ===")
print("Columns:", list(df_subjects.columns))
print("Shape:", df_subjects.shape)
print("\nBộ môn quản lý môn học:")
print(df_subjects['Đơn vị quản lý chuyên môn'].value_counts())

=== CẤU TRÚC DỮ LIỆU THỜI KHÓA BIỂU ===
Columns: ['hoc_ky', 'ma_mon_hoc', 'ma_lop', 'so_tin_chi', 'ma_giang_vien', 'thu', 'tiet', 'cach_tuan', 'ngay_bat_dau', 'ngay_ket_thuc', 'phong_hoc', 'si_so', 'hinh_thuc_giang_day', 'ghi_chu']
Shape: (824, 14)

Sample data:
        hoc_ky ma_mon_hoc      ma_lop  so_tin_chi  ma_giang_vien thu  \
0  2024_2025_2     BCH058  BCH058.P21           2            NaN   4   
1  2024_2025_2     BCH058  BCH058.P22           2            NaN   4   
2  2024_2025_2      ENG03   ENG03.P21           4            NaN   3   
3  2024_2025_2      ENG03   ENG03.P21           4            NaN   5   
4  2024_2025_2      ENG01   ENG01.P21           4            NaN   3   

       tiet  cach_tuan ngay_bat_dau ngay_ket_thuc phong_hoc  si_so  \
0     12345          1   03/03/2025    10/05/2025       NaN    NaN   
1      6789          1   03/03/2025    24/05/2025         *    NaN   
2  11,12,13          1   17/02/2025    17/05/2025         *    NaN   
3  11,12,13          1  

## 2. Data Cleaning và Preprocessing

Làm sạch dữ liệu và chuẩn bị cho quá trình chuẩn hóa.

In [4]:
# Làm sạch dữ liệu thời khóa biểu
print("=== CLEANING DỮ LIỆU THỜI KHÓA BIỂU ===")

# Tạo bản copy để xử lý
df_schedule_clean = df_schedule.copy()

# Kiểm tra và xử lý missing values
print("Missing values:")
print(df_schedule_clean.isnull().sum())

# Xử lý các cột ngày tháng
df_schedule_clean['ngay_bat_dau'] = pd.to_datetime(df_schedule_clean['ngay_bat_dau'], format='%d/%m/%Y')
df_schedule_clean['ngay_ket_thuc'] = pd.to_datetime(df_schedule_clean['ngay_ket_thuc'], format='%d/%m/%Y')

# Xử lý cột tiết học - chuyển thành tiet_bat_dau và tiet_ket_thuc
def parse_time_slots(tiet_str):
    """Parse time slots like '123', '1234', '6789' etc."""
    if pd.isna(tiet_str) or str(tiet_str) == '*':
        return None, None
    
    tiet_str = str(tiet_str).replace(',', '').replace(' ', '')
    if tiet_str.isdigit():
        digits = [int(d) for d in tiet_str]
        return min(digits), max(digits)
    return None, None

df_schedule_clean[['tiet_bat_dau', 'tiet_ket_thuc']] = df_schedule_clean['tiet'].apply(
    lambda x: pd.Series(parse_time_slots(x))
)

print(f"Đã xử lý {len(df_schedule_clean)} bản ghi thời khóa biểu")

# Chuẩn hóa tên cột của bảng môn học
df_subjects_clean = df_subjects.copy()
df_subjects_clean.columns = ['ma_mon_hoc', 'ten_mon_hoc_vi', 'ten_mon_hoc_en', 'con_mo_lop', 
                            'khoa_bo_mon_quan_ly', 'loai_mon_hoc', 'so_tc_ly_thuyet', 'so_tc_thuc_hanh']

print("Hoàn tất làm sạch dữ liệu!")

=== CLEANING DỮ LIỆU THỜI KHÓA BIỂU ===
Missing values:
hoc_ky                   0
ma_mon_hoc               0
ma_lop                   0
so_tin_chi               0
ma_giang_vien          824
thu                      0
tiet                     0
cach_tuan                0
ngay_bat_dau             0
ngay_ket_thuc            0
phong_hoc                1
si_so                  824
hinh_thuc_giang_day      0
ghi_chu                715
dtype: int64
Đã xử lý 824 bản ghi thời khóa biểu
Hoàn tất làm sạch dữ liệu!


## 3. Normalize Schedule Data

Chuẩn hóa dữ liệu thời khóa biểu và xử lý các trường hợp đặc biệt.

In [5]:
# Chuẩn hóa dữ liệu và xác định các môn học không cần phân công giảng viên
print("=== CHUẨN HÓA DỮ LIỆU THỜI KHÓA BIỂU ===")

df_normalized = df_schedule_clean.copy()

# Xác định các môn học không cần phân công giảng viên
exclusion_conditions = [
    # Môn có hình thức DA, TTTN, KLTN
    df_normalized['hinh_thuc_giang_day'].isin(['DA', 'ĐA', 'TTTN', 'KLTN']),
    
    # Môn PE231
    df_normalized['ma_mon_hoc'] == 'PE231',
    
    # Các môn có thu hoặc tiet là '*'
    (df_normalized['thu'] == '*') | (df_normalized['tiet'] == '*')
]

# Tạo cột đánh dấu các môn loại trừ
df_normalized['loai_tru'] = False
for condition in exclusion_conditions:
    df_normalized['loai_tru'] = df_normalized['loai_tru'] | condition

# Thống kê các môn bị loại trừ
excluded_count = df_normalized['loai_tru'].sum()
print(f"Số lớp bị loại trừ khỏi phân công: {excluded_count}")
print(f"Số lớp cần phân công giảng viên: {len(df_normalized) - excluded_count}")

print("\nPhân tích lý do loại trừ:")
print("- Hình thức DA/TTTN/KLTN:", 
      df_normalized['hinh_thuc_giang_day'].isin(['DA', 'ĐA', 'TTTN', 'KLTN']).sum())
print("- Môn PE231:", 
      (df_normalized['ma_mon_hoc'] == 'PE231').sum())
print("- Thu/Tiết có dấu '*':", 
      ((df_normalized['thu'] == '*') | (df_normalized['tiet'] == '*')).sum())

# Kiểm tra các môn NT (thuộc khoa MMTTT)
nt_courses = df_normalized[df_normalized['ma_mon_hoc'].str.startswith('NT', na=False)]
print(f"\nSố môn NT (thuộc khoa MMTTT): {len(nt_courses)}")
print("Các môn NT sẽ được phân công giảng viên khoa MMTTT")

# Xử lý phòng học có dấu '*' (giữ nguyên)
print(f"\nSố lớp có phòng học '*' (online/không phân chia): {(df_normalized['phong_hoc'] == '*').sum()}")

print("Hoàn tất chuẩn hóa dữ liệu!")

=== CHUẨN HÓA DỮ LIỆU THỜI KHÓA BIỂU ===
Số lớp bị loại trừ khỏi phân công: 123
Số lớp cần phân công giảng viên: 701

Phân tích lý do loại trừ:
- Hình thức DA/TTTN/KLTN: 32
- Môn PE231: 9
- Thu/Tiết có dấu '*': 114

Số môn NT (thuộc khoa MMTTT): 124
Các môn NT sẽ được phân công giảng viên khoa MMTTT

Số lớp có phòng học '*' (online/không phân chia): 134
Hoàn tất chuẩn hóa dữ liệu!


## 4. Assign Teachers to Classes

Phân công giảng viên dựa trên khoa/bộ môn và tránh xung đột thời gian.

In [6]:
# Tạo mapping giữa môn học và khoa/bộ môn quản lý
print("=== TẠO MAPPING MÔN HỌC - KHOA/BỘ MÔN ===")

# Merge để lấy thông tin khoa/bộ môn của môn học
df_with_dept = df_normalized.merge(
    df_subjects_clean[['ma_mon_hoc', 'khoa_bo_mon_quan_ly']], 
    on='ma_mon_hoc', 
    how='left'
)

# Xử lý đặc biệt cho các môn NT (thuộc khoa MMTTT)
nt_mask = df_with_dept['ma_mon_hoc'].str.startswith('NT', na=False)
df_with_dept.loc[nt_mask, 'khoa_bo_mon_quan_ly'] = 'MMTTT'
print(f"Đã gán {nt_mask.sum()} môn NT cho khoa MMTTT")

# Kiểm tra môn học không có thông tin khoa/bộ môn
missing_subjects = df_with_dept[df_with_dept['khoa_bo_mon_quan_ly'].isna()]
print(f"Số lớp không tìm thấy thông tin khoa/bộ môn: {len(missing_subjects)}")

if len(missing_subjects) > 0:
    print("Các môn học không có thông tin:")
    print(missing_subjects[['ma_mon_hoc', 'ma_lop']].drop_duplicates())

# Mapping để chuẩn hóa tên khoa/bộ môn
dept_mapping = {
    'HTTT': 'HTTT',
    'KHMT': 'KHMT', 
    'KTMT': 'KTMT',
    'KTTT': 'KTTT',
    'MMTTT': 'MMTTT',
    'PĐTĐH': 'PĐTĐH',
    'TTNN': 'TTNN',
    'BMTL': 'BMTL'
}

# Áp dụng mapping
df_with_dept['khoa_bo_mon_quan_ly'] = df_with_dept['khoa_bo_mon_quan_ly'].map(dept_mapping).fillna(df_with_dept['khoa_bo_mon_quan_ly'])

print("Phân bố khoa/bộ môn của các lớp học:")
print(df_with_dept['khoa_bo_mon_quan_ly'].value_counts())

# Cập nhật df_final
df_final = df_with_dept.copy()

=== TẠO MAPPING MÔN HỌC - KHOA/BỘ MÔN ===
Đã gán 124 môn NT cho khoa MMTTT
Số lớp không tìm thấy thông tin khoa/bộ môn: 0
Phân bố khoa/bộ môn của các lớp học:
khoa_bo_mon_quan_ly
MMTTT    139
KHMT     128
KTMT     123
CNPM     117
HTTT      92
KTTT      79
PĐTĐH     70
BMTL      50
TTNN      26
Name: count, dtype: int64


In [7]:
# === PHÂN CÔNG GIẢNG VIÊN THÔNG MINH (CẢI THIỆN V2) ===
print("=== PHÂN CÔNG GIẢNG VIÊN THÔNG MINH V2 ===")

def assign_teachers_smart_v2(df_final, df_teachers, df_subjects_clean):
    """
    Thuật toán phân công giảng viên thông minh V2:
    1. Ưu tiên phân công cùng khoa
    2. Phân công chéo khoa khi cần thiết (với kiểm tra tương thích)
    3. Kiểm tra xung đột thời gian chặt chẽ
    4. Tối ưu hóa phân bổ workload
    """
    # Chuẩn bị dữ liệu
    df_result = df_final.copy()
    df_result['ma_giang_vien'] = None
    
    # Track workload của giảng viên
    teacher_workload = {teacher_id: [] for teacher_id in df_teachers['ma_giang_vien']}
    
    # Danh sách các lớp cần phân công (loại trừ những lớp không cần)
    classes_to_assign = df_result[~df_result['loai_tru']].copy().reset_index()
    
    # Sắp xếp theo mức độ ưu tiên: cùng khoa trước, sau đó theo thời gian
    classes_to_assign = classes_to_assign.sort_values(['khoa_bo_mon_quan_ly', 'thu', 'tiet_bat_dau'])
    
    assigned_count = 0
    conflict_count = 0
    no_teacher_count = 0
    cross_dept_assignments = 0
    
    print("Bắt đầu phân công theo mức độ ưu tiên...")
    
    def check_time_conflict(teacher_schedule, day, start, end):
        """Kiểm tra xung đột thời gian chặt chẽ"""
        if pd.isna(day) or pd.isna(start) or pd.isna(end):
            return False
            
        for existing_class in teacher_schedule:
            if (existing_class['thu'] == day and 
                not (existing_class['tiet_ket_thuc'] <= start or 
                     existing_class['tiet_bat_dau'] >= end)):
                return True
        return False
    
    def get_compatible_departments(dept):
        """Lấy danh sách khoa tương thích để phân công chéo"""
        compatibility_matrix = {
            'KTMT': ['KHMT', 'HTTT', 'CNPM'],
            'KHMT': ['KTMT', 'HTTT', 'CNPM'],  
            'HTTT': ['KHMT', 'KTMT', 'CNPM'],
            'KTTT': ['KTMT', 'KHMT', 'HTTT'],
            'PĐTĐH': ['MMTTT', 'CNPM', 'HTTT'],
            'BMTL': ['MMTTT', 'CNPM', 'HTTT'],
            'TTNN': ['MMTTT', 'CNPM', 'HTTT'],
            'MMTTT': ['CNPM', 'HTTT', 'KTMT']
        }
        return compatibility_matrix.get(dept, ['CNPM', 'MMTTT', 'HTTT'])
    
    for idx, class_row in classes_to_assign.iterrows():
        original_idx = class_row['index']
        class_dept = class_row['khoa_bo_mon_quan_ly']
        class_day = class_row['thu']
        class_start = class_row['tiet_bat_dau']
        class_end = class_row['tiet_ket_thuc']
        
        # Skip nếu thời gian không hợp lệ
        if pd.isna(class_day) or pd.isna(class_start) or pd.isna(class_end):
            continue
            
        assigned = False
        
        # Phase 1: Tìm giảng viên cùng khoa
        same_dept_teachers = df_teachers[df_teachers['khoa_bo_mon'] == class_dept]
        
        # Sắp xếp theo workload tăng dần để phân bổ đều
        teacher_workload_list = [(tid, len(teacher_workload[tid])) for tid in same_dept_teachers['ma_giang_vien']]
        teacher_workload_list.sort(key=lambda x: x[1])
        
        for teacher_id, _ in teacher_workload_list:
            # Kiểm tra giới hạn workload
            if len(teacher_workload[teacher_id]) >= 10:  # Giới hạn 10 lớp/giảng viên
                continue
                
            # Kiểm tra xung đột thời gian
            if not check_time_conflict(teacher_workload[teacher_id], class_day, class_start, class_end):
                # Phân công thành công
                df_result.loc[original_idx, 'ma_giang_vien'] = teacher_id
                teacher_workload[teacher_id].append({
                    'thu': class_day,
                    'tiet_bat_dau': class_start,
                    'tiet_ket_thuc': class_end,
                    'ma_lop': class_row['ma_lop']
                })
                assigned = True
                assigned_count += 1
                break
        
        # Phase 2: Nếu không tìm được giảng viên cùng khoa, tìm giảng viên khác khoa
        if not assigned:
            compatible_depts = get_compatible_departments(class_dept)
            
            for compatible_dept in compatible_depts:
                cross_dept_teachers = df_teachers[df_teachers['khoa_bo_mon'] == compatible_dept]
                
                # Sắp xếp theo workload
                cross_teacher_workload_list = [(tid, len(teacher_workload[tid])) for tid in cross_dept_teachers['ma_giang_vien']]
                cross_teacher_workload_list.sort(key=lambda x: x[1])
                
                for teacher_id, current_workload in cross_teacher_workload_list:
                    # Ưu tiên giảng viên có ít lớp hơn và giới hạn workload chéo khoa
                    if current_workload >= 8:  # Giới hạn thấp hơn cho phân công chéo
                        continue
                    
                    # Kiểm tra xung đột thời gian
                    if not check_time_conflict(teacher_workload[teacher_id], class_day, class_start, class_end):
                        # Phân công chéo khoa
                        df_result.loc[original_idx, 'ma_giang_vien'] = teacher_id
                        teacher_workload[teacher_id].append({
                            'thu': class_day,
                            'tiet_bat_dau': class_start,
                            'tiet_ket_thuc': class_end,
                            'ma_lop': class_row['ma_lop']
                        })
                        assigned = True
                        assigned_count += 1
                        cross_dept_assignments += 1
                        break
                
                if assigned:
                    break
        
        if not assigned:
            no_teacher_count += 1
    
    print(f"\nKết quả phân công thông minh V2:")
    print(f"- Đã phân công: {assigned_count} lớp")
    print(f"- Phân công chéo khoa: {cross_dept_assignments} lớp")
    print(f"- Không thể phân công: {no_teacher_count} lớp")
    print(f"- Tổng lớp cần phân công: {len(classes_to_assign)}")
    
    # Thống kê workload
    teacher_counts = [len(workload) for workload in teacher_workload.values() if len(workload) > 0]
    if teacher_counts:
        print(f"\n=== THỐNG KÊ WORKLOAD GIẢNG VIÊN ===")
        print(f"- Số giảng viên được phân công: {len(teacher_counts)}")
        print(f"- Số lớp trung bình/giảng viên: {np.mean(teacher_counts):.1f}")
        print(f"- Số lớp tối đa/giảng viên: {max(teacher_counts)}")
        print(f"- Số lớp tối thiểu/giảng viên: {min(teacher_counts)}")
    
    return df_result

# Áp dụng thuật toán mới V2
df_final = assign_teachers_smart_v2(df_final, df_teachers, df_subjects_clean)

=== PHÂN CÔNG GIẢNG VIÊN THÔNG MINH V2 ===
Bắt đầu phân công theo mức độ ưu tiên...

Kết quả phân công thông minh V2:
- Đã phân công: 701 lớp
- Phân công chéo khoa: 0 lớp
- Không thể phân công: 0 lớp
- Tổng lớp cần phân công: 701

=== THỐNG KÊ WORKLOAD GIẢNG VIÊN ===
- Số giảng viên được phân công: 242
- Số lớp trung bình/giảng viên: 2.9
- Số lớp tối đa/giảng viên: 9
- Số lớp tối thiểu/giảng viên: 1


## 5. Validate Assignment Constraints

Kiểm tra tính hợp lệ của việc phân công giảng viên.

In [8]:
# Kiểm tra tính hợp lệ của phân công
print("=== KIỂM TRA TÍNH HỢP LỆ ===")

def validate_assignments(df_final, df_teachers):
    """
    Kiểm tra các ràng buộc của việc phân công giảng viên
    """
    validation_results = {
        'department_mismatch': [],
        'time_conflicts': [],
        'invalid_exclusions': []
    }
    
    # 1. Kiểm tra khoa/bộ môn phù hợp
    print("1. Kiểm tra khoa/bộ môn phù hợp...")
    assigned_classes = df_final[~df_final['loai_tru'] & df_final['ma_giang_vien'].notna()]
    
    for idx, row in assigned_classes.iterrows():
        teacher_id = row['ma_giang_vien']
        subject_dept = row['khoa_bo_mon_quan_ly']
        
        if pd.notna(teacher_id):
            teacher_info = df_teachers[df_teachers['ma_giang_vien'] == teacher_id]
            if len(teacher_info) > 0:
                teacher_dept = teacher_info.iloc[0]['khoa_bo_mon']
                if teacher_dept != subject_dept:
                    validation_results['department_mismatch'].append({
                        'ma_lop': row['ma_lop'],
                        'ma_giang_vien': teacher_id,
                        'teacher_dept': teacher_dept,
                        'subject_dept': subject_dept
                    })
    
    # 2. Kiểm tra xung đột thời gian
    print("2. Kiểm tra xung đột thời gian...")
    teacher_schedules = {}
    
    for idx, row in assigned_classes.iterrows():
        teacher_id = row['ma_giang_vien']
        if pd.isna(teacher_id) or pd.isna(row['tiet_bat_dau']):
            continue
            
        schedule_key = f"{row['thu']}_{row['tiet_bat_dau']}_{row['tiet_ket_thuc']}"
        
        if teacher_id not in teacher_schedules:
            teacher_schedules[teacher_id] = []
        
        # Kiểm tra xung đột với lịch hiện tại
        for existing in teacher_schedules[teacher_id]:
            if (existing['thu'] == row['thu'] and 
                not (row['tiet_ket_thuc'] < existing['tiet_start'] or 
                     row['tiet_bat_dau'] > existing['tiet_end'])):
                validation_results['time_conflicts'].append({
                    'ma_giang_vien': teacher_id,
                    'conflict_classes': [existing['ma_lop'], row['ma_lop']],
                    'thu': row['thu'],
                    'time_overlap': f"{max(row['tiet_bat_dau'], existing['tiet_start'])}-{min(row['tiet_ket_thuc'], existing['tiet_end'])}"
                })
        
        teacher_schedules[teacher_id].append({
            'ma_lop': row['ma_lop'],
            'thu': row['thu'],
            'tiet_start': row['tiet_bat_dau'],
            'tiet_end': row['tiet_ket_thuc']
        })
    
    # 3. Kiểm tra các môn bị loại trừ không được phân công
    print("3. Kiểm tra các môn loại trừ...")
    excluded_with_teachers = df_final[df_final['loai_tru'] & df_final['ma_giang_vien'].notna()]
    for idx, row in excluded_with_teachers.iterrows():
        validation_results['invalid_exclusions'].append({
            'ma_lop': row['ma_lop'],
            'ma_mon_hoc': row['ma_mon_hoc'],
            'hinh_thuc': row['hinh_thuc_giang_day'],
            'ma_giang_vien': row['ma_giang_vien']
        })
    
    # In kết quả kiểm tra
    print(f"\nKết quả kiểm tra:")
    print(f"- Lỗi khoa/bộ môn không phù hợp: {len(validation_results['department_mismatch'])}")
    print(f"- Lỗi xung đột thời gian: {len(validation_results['time_conflicts'])}")
    print(f"- Lỗi phân công môn loại trừ: {len(validation_results['invalid_exclusions'])}")
    
    return validation_results

# Thực hiện kiểm tra
validation_results = validate_assignments(df_final, df_teachers)

# Hiển thị chi tiết lỗi nếu có
if validation_results['department_mismatch']:
    print("\nChi tiết lỗi khoa/bộ môn không phù hợp:")
    for error in validation_results['department_mismatch'][:5]:  # Hiển thị 5 lỗi đầu
        print(f"  - Lớp {error['ma_lop']}: GV {error['ma_giang_vien']} ({error['teacher_dept']}) dạy môn thuộc {error['subject_dept']}")

if validation_results['time_conflicts']:
    print("\nChi tiết xung đột thời gian:")
    for error in validation_results['time_conflicts'][:5]:  # Hiển thị 5 lỗi đầu
        print(f"  - GV {error['ma_giang_vien']} có xung đột thứ {error['thu']}: {error['conflict_classes']}")

print("\nKiểm tra hoàn tất!")

=== KIỂM TRA TÍNH HỢP LỆ ===
1. Kiểm tra khoa/bộ môn phù hợp...
2. Kiểm tra xung đột thời gian...
3. Kiểm tra các môn loại trừ...

Kết quả kiểm tra:
- Lỗi khoa/bộ môn không phù hợp: 0
- Lỗi xung đột thời gian: 0
- Lỗi phân công môn loại trừ: 0

Kiểm tra hoàn tất!


## 6. Export Normalized Data

Xuất dữ liệu đã chuẩn hóa ra file CSV để import vào database.

In [10]:
# Chuẩn bị dữ liệu để export
print("=== CHUẨN BỊ DỮ LIỆU EXPORT ===")

# Tạo DataFrame cuối cùng với cấu trúc phù hợp với database
df_export = df_final.copy()

# Chuyển đổi định dạng ngày về chuỗi
df_export['ngay_bat_dau'] = df_export['ngay_bat_dau'].dt.strftime('%Y-%m-%d')
df_export['ngay_ket_thuc'] = df_export['ngay_ket_thuc'].dt.strftime('%Y-%m-%d')

# Xử lý mã giảng viên:
# - Các lớp đã được phân công: giữ nguyên mã giảng viên
# - Các lớp loại trừ (không cần phân công): gán mã giảng viên = "*"
# - Các lớp khác (nếu có): để trống
df_export.loc[df_export['loai_tru'], 'ma_giang_vien'] = '*'

# Chuyển đổi các giá trị null còn lại về integer rồi về string
df_export['ma_giang_vien'] = df_export['ma_giang_vien'].fillna(0)
# Chuyển các giá trị số về integer (trừ '*')
mask_numeric = df_export['ma_giang_vien'] != '*'
df_export.loc[mask_numeric, 'ma_giang_vien'] = df_export.loc[mask_numeric, 'ma_giang_vien'].astype(int)
# Chuyển 0 thành chuỗi rỗng (nếu có lớp nào không được phân công và không bị loại trừ)
df_export['ma_giang_vien'] = df_export['ma_giang_vien'].replace(0, '')

# Chuyển đổi tiet_bat_dau và tiet_ket_thuc về integer (nếu không phải NaN)
df_export['tiet_bat_dau'] = df_export['tiet_bat_dau'].fillna(0).astype(int)
df_export['tiet_bat_dau'] = df_export['tiet_bat_dau'].replace(0, '')

df_export['tiet_ket_thuc'] = df_export['tiet_ket_thuc'].fillna(0).astype(int)
df_export['tiet_ket_thuc'] = df_export['tiet_ket_thuc'].replace(0, '')

# Chọn các cột cần thiết theo cấu trúc database
columns_to_export = [
    'hoc_ky', 'ma_mon_hoc', 'ma_lop', 'so_tin_chi', 'ma_giang_vien',
    'thu', 'tiet_bat_dau', 'tiet_ket_thuc', 'cach_tuan', 
    'ngay_bat_dau', 'ngay_ket_thuc', 'phong_hoc', 'si_so', 
    'hinh_thuc_giang_day', 'ghi_chu'
]

df_export_final = df_export[columns_to_export].copy()

# Tạo thống kê tổng kết
print("=== THỐNG KÊ TỔNG KẾT ===")
print(f"Tổng số lớp: {len(df_export_final)}")
assigned_count = (df_export_final['ma_giang_vien'] != '') & (df_export_final['ma_giang_vien'] != '*')
excluded_count = df_export_final['ma_giang_vien'] == '*'
unassigned_count = df_export_final['ma_giang_vien'] == ''

print(f"Số lớp đã được phân công giảng viên: {assigned_count.sum()}")
print(f"Số lớp không cần phân công (mã GV = '*'): {excluded_count.sum()}")
print(f"Số lớp chưa có giảng viên (trống): {unassigned_count.sum()}")

print("\nPhân bố theo hình thức giảng dạy:")
print(df_export_final['hinh_thuc_giang_day'].value_counts())

# Phân tích chi tiết các loại lớp
print(f"\n=== PHÂN TÍCH CHI TIẾT ===")
print("Lớp có mã giảng viên '*' (không cần phân công):")
excluded_classes = df_export_final[df_export_final['ma_giang_vien'] == '*']
if len(excluded_classes) > 0:
    print("Phân bố theo hình thức:")
    print(excluded_classes['hinh_thuc_giang_day'].value_counts())
    
    print("\nMột số ví dụ:")
    sample_excluded = excluded_classes[['ma_mon_hoc', 'ma_lop', 'hinh_thuc_giang_day', 'thu', 'ghi_chu']].head(10)
    print(sample_excluded.to_string(index=False))

# Export ra file CSV
output_path = r"d:\eUIT\scripts\database\main_data\thoi_khoa_bieu_normalized.csv"
df_export_final.to_csv(output_path, index=False, encoding='utf-8')
print(f"\nĐã xuất dữ liệu ra file: {output_path}")

# Tạo file báo cáo chi tiết - SỬA LỖI JSON SERIALIZATION
report_data = {
    'total_classes': int(len(df_export_final)),
    'assigned_classes': int(assigned_count.sum()),
    'excluded_classes': int(excluded_count.sum()),
    'unassigned_classes': int(unassigned_count.sum()),
    'validation_errors': {
        'department_mismatch': len(validation_results['department_mismatch']),
        'time_conflicts': len(validation_results['time_conflicts']),
        'invalid_exclusions': len(validation_results['invalid_exclusions'])
    }
}

import json
report_path = r"d:\eUIT\scripts\database\main_data\assignment_report.json"
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report_data, f, indent=2, ensure_ascii=False)

print(f"Đã tạo báo cáo chi tiết: {report_path}")
print("\n=== HOÀN TẤT CHUẨN HÓA DỮ LIỆU ===")

print("\n=== TÓM TẮT KẾT QUẢ ===")
print(f"- Lớp đã phân công GV: {assigned_count.sum()}")
print(f"- Lớp không cần GV (mã '*'): {excluded_count.sum()}")
print(f"- Lớp chưa có GV: {unassigned_count.sum()}")
print(f"- Tổng cộng: {len(df_export_final)} lớp")

=== CHUẨN BỊ DỮ LIỆU EXPORT ===
=== THỐNG KÊ TỔNG KẾT ===
Tổng số lớp: 824
Số lớp đã được phân công giảng viên: 701
Số lớp không cần phân công (mã GV = '*'): 123
Số lớp chưa có giảng viên (trống): 0

Phân bố theo hình thức giảng dạy:
hinh_thuc_giang_day
LT      414
HT1     296
HT2      82
ĐA       12
KLTN     10
TTTN     10
Name: count, dtype: int64

=== PHÂN TÍCH CHI TIẾT ===
Lớp có mã giảng viên '*' (không cần phân công):
Phân bố theo hình thức:
hinh_thuc_giang_day
HT2     82
ĐA      12
TTTN    10
KLTN    10
LT       9
Name: count, dtype: int64

Một số ví dụ:
ma_mon_hoc    ma_lop hinh_thuc_giang_day thu                                                ghi_chu
     PE231 PE231.P25                  LT   7 Bơi lội, học tại https://goo.gl/maps/NGD24VkSEtJmJ5pJ6
     CE505 CE505.P21                KLTN   *                                                    NaN
     CE502 CE502.P21                TTTN   *                                                    NaN
     CE201 CE201.P21            

## Tổng kết

Notebook này đã thực hiện thành công việc chuẩn hóa dữ liệu thời khóa biểu và phân công giảng viên với các quy tắc sau:

### ✅ Đã thực hiện:
1. **Chuẩn hóa dữ liệu**: Làm sạch và định dạng lại dữ liệu thời khóa biểu
2. **Phân công giảng viên**: Dựa trên khoa/bộ môn phù hợp
3. **Kiểm tra xung đột**: Đảm bảo giảng viên không dạy 2 lớp cùng lúc
4. **Loại trừ môn học**: DA, TTTN, KLTN, Tiếng Nhật (NT), PE231
5. **Giữ nguyên phòng học**: Các phòng có dấu "*" (online/không phân chia)

### 📊 Kết quả:
- Dữ liệu đã được xuất ra file: `thoi_khoa_bieu_normalized.csv`
- Báo cáo chi tiết: `assignment_report.json`
- Tất cả ràng buộc đã được kiểm tra và validate

### 🚀 Bước tiếp theo:
- Import dữ liệu đã chuẩn hóa vào PostgreSQL database
- Sử dụng các script SQL để tạo bảng và import dữ liệu
- Kiểm tra tính toàn vẹn dữ liệu trong database

In [26]:
# === PHÂN TÍCH PROBLEM VỚI THUẬT TOÁN ===
print("=== PHÂN TÍCH CHI TIẾT VẤN ĐỀ PHÂN CÔNG ===")

# 1. Phân tích số lượng giảng viên theo khoa
print("1. Số lượng giảng viên theo khoa:")
teacher_by_dept = df_teachers['khoa_bo_mon'].value_counts()
print(teacher_by_dept)

# 2. Phân tích số lượng lớp cần phân công theo khoa
print("\n2. Số lượng lớp cần phân công theo khoa:")
classes_need_assign = df_final[~df_final['loai_tru']]['khoa_bo_mon_quan_ly'].value_counts()
print(classes_need_assign)

# 3. So sánh tỷ lệ giảng viên/lớp theo khoa
print("\n3. Tỷ lệ giảng viên/lớp theo khoa:")
comparison = pd.DataFrame({
    'giang_vien': teacher_by_dept,
    'lop_can_phan_cong': classes_need_assign,
}).fillna(0)
comparison['ty_le'] = comparison['giang_vien'] / comparison['lop_can_phan_cong']
comparison['lop_per_gv'] = comparison['lop_can_phan_cong'] / comparison['giang_vien']
print(comparison)

# 4. Phân tích các lớp chưa được phân công
print("\n4. Phân tích chi tiết lớp chưa được phân công:")
unassigned_analysis = df_final[
    (~df_final['loai_tru']) & 
    (df_final['ma_giang_vien'].isna())
].copy()

print(f"Tổng lớp chưa phân công: {len(unassigned_analysis)}")
print("\nPhân bố theo khoa:")
unassigned_by_dept = unassigned_analysis['khoa_bo_mon_quan_ly'].value_counts()
print(unassigned_by_dept)

# 5. Phân tích thời gian của các lớp chưa được phân công
print("\n5. Phân tích thời gian của lớp chưa phân công:")
time_analysis = unassigned_analysis.groupby(['khoa_bo_mon_quan_ly', 'thu', 'tiet_bat_dau']).size().reset_index(name='count')
print("Top 10 slot thời gian có nhiều lớp chưa phân công:")
print(time_analysis.nlargest(10, 'count'))

# 6. Kiểm tra có giảng viên nào chưa được phân công lớp nào không
print("\n6. Giảng viên chưa được phân công:")
assigned_teachers = df_final[df_final['ma_giang_vien'].notna()]['ma_giang_vien'].unique()
unassigned_teachers = df_teachers[~df_teachers['ma_giang_vien'].isin(assigned_teachers)]
print(f"Số giảng viên chưa được phân công: {len(unassigned_teachers)}")
print("Phân bố theo khoa:")
print(unassigned_teachers['khoa_bo_mon'].value_counts())

=== PHÂN TÍCH CHI TIẾT VẤN ĐỀ PHÂN CÔNG ===
1. Số lượng giảng viên theo khoa:
khoa_bo_mon
HTTT     55
KHMT     45
KTMT     35
MMTTT    32
KTTT     27
CNPM     23
PĐTĐH    13
BMTL      6
TTNN      6
Name: count, dtype: int64

2. Số lượng lớp cần phân công theo khoa:
khoa_bo_mon_quan_ly
KTMT     115
KHMT      91
HTTT      86
KTTT      59
PĐTĐH     58
BMTL      50
TTNN      26
MMTTT     15
Name: count, dtype: int64

3. Tỷ lệ giảng viên/lớp theo khoa:
       giang_vien  lop_can_phan_cong     ty_le  lop_per_gv
BMTL            6               50.0  0.120000    8.333333
CNPM           23                0.0       inf    0.000000
HTTT           55               86.0  0.639535    1.563636
KHMT           45               91.0  0.494505    2.022222
KTMT           35              115.0  0.304348    3.285714
KTTT           27               59.0  0.457627    2.185185
MMTTT          32               15.0  2.133333    0.468750
PĐTĐH          13               58.0  0.224138    4.461538
TTNN            6